In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import re

In [2]:
data=pd.read_csv(r"C:/Users/Dell/Pictures/senti/initialdata.csv")
initial=pd.read_csv(r"C:/Users/Dell/Pictures/senti/initialdata.csv")

#data['Review'] = data['title'].str.cat(data['content'], sep =". ") 
#initial=data[['Review']]
#initial.head()
#initial.to_csv("C:/Users/Dell/Pictures/senti/initialdata.csv")


In [3]:
data.head()

,row_index,Review
0,0,Noting Special. Ok Product.
1,1,Review. Good
2,2,"Indian brand keeping to its name, best for me!..."
3,3,Satisfied. Bought it for INR 152 during the Am...
4,4,Damaged. Unexpectedlly... One bottle is damaged


In [4]:
#Using spaCy for dependency parsing which forms the crux of aspect extraction
import spacy
from tqdm import tqdm
nlp = spacy.load('C:/Users/Dell/Anaconda3/Lib/site-packages/en_core_web_lg/en_core_web_lg-2.1.0', parse=True, tag=True, entity=True)

In [5]:
#COMPOUND WORDS
competitors = ['marico','P&G','hindustan unilever limited','dabur','petra foods','nestle'] 

In [6]:
aspect_terms = []
comp_terms = []
easpect_terms = []
ecomp_terms = []
enemy = []

def get_aspect_adj(data):
    for x in tqdm(range(len(data['Review']))):
        amod_pairs = []
        advmod_pairs = []
        compound_pairs = []
        xcomp_pairs = []
        neg_pairs = []
        eamod_pairs = []
        eadvmod_pairs = []
        ecompound_pairs = []
        eneg_pairs = []
        excomp_pairs = []
        enemlist = []
        if len(str(data['Review'][x])) != 0:
            lines = str(data['Review'][x]).replace('*',' ').replace('-',' ').replace('so ',' ').replace('be ',' ').replace('are ',' ').replace('just ',' ').replace('get ','').replace('were ',' ').replace('When ','').replace('when ','').replace('again ',' ').replace('where ','').replace('how ',' ').replace('has ',' ').replace('Here ',' ').replace('here ',' ').replace('now ',' ').replace('see ',' ').replace('why ',' ').split('.')       
            for line in lines:
                enem_list = []
                for eny in competitors:
                    enem = re.search(eny,line)
                    if enem is not None:
                        enem_list.append(enem.group())
                if len(enem_list)==0:
                    doc = nlp(line)
                    str1=''
                    str2=''
                    for token in doc:
                        if token.pos_ is 'NOUN':
                            for j in token.lefts:
                                if j.dep_ == 'compound':
                                    compound_pairs.append((j.text+' '+token.text,token.text))
                                if j.dep_ is 'amod' and j.pos_ is 'ADJ': #primary condition
                                    str1 = j.text+' '+token.text
                                    amod_pairs.append(j.text+' '+token.text)
                                    for k in j.lefts:
                                        if k.dep_ is 'advmod': #secondary condition to get adjective of adjectives
                                            str2 = k.text+' '+j.text+' '+token.text
                                            amod_pairs.append(k.text+' '+j.text+' '+token.text)
                                    mtch = re.search(re.escape(str1),re.escape(str2))
                                    if mtch is not None:
                                        amod_pairs.remove(str1)
                        if token.pos_ is 'VERB':
                            for j in token.lefts:
                                if j.dep_ is 'advmod' and j.pos_ is 'ADV':
                                    advmod_pairs.append(j.text+' '+token.text)
                                if j.dep_ is 'neg' and j.pos_ is 'ADV':
                                    neg_pairs.append(j.text+' '+token.text)
                            for j in token.rights:
                                if j.dep_ is 'advmod'and j.pos_ is 'ADV':
                                    advmod_pairs.append(token.text+' '+j.text)
                        if token.pos_ is 'ADJ':
                            for j,h in zip(token.rights,token.lefts):
                                if j.dep_ is 'xcomp' and h.dep_ is not 'neg':
                                    for k in j.lefts:
                                        if k.dep_ is 'aux':
                                            xcomp_pairs.append(token.text+' '+k.text+' '+j.text)
                                elif j.dep_ is 'xcomp' and h.dep_ is 'neg':
                                    if k.dep_ is 'aux':
                                            neg_pairs.append(h.text +' '+token.text+' '+k.text+' '+j.text)

                else:
                    enemlist.append(enem_list)
                    doc = nlp(line)
                    str1=''
                    str2=''
                    for token in doc:
                        if token.pos_ is 'NOUN':
                            for j in token.lefts:
                                if j.dep_ == 'compound':
                                    ecompound_pairs.append((j.text+' '+token.text,token.text))
                                if j.dep_ is 'amod' and j.pos_ is 'ADJ': #primary condition
                                    str1 = j.text+' '+token.text
                                    eamod_pairs.append(j.text+' '+token.text)
                                    for k in j.lefts:
                                        if k.dep_ is 'advmod': #secondary condition to get adjective of adjectives
                                            str2 = k.text+' '+j.text+' '+token.text
                                            eamod_pairs.append(k.text+' '+j.text+' '+token.text)
                                    mtch = re.search(re.escape(str1),re.escape(str2))
                                    if mtch is not None:
                                        eamod_pairs.remove(str1)
                        if token.pos_ is 'VERB':
                            for j in token.lefts:
                                if j.dep_ is 'advmod' and j.pos_ is 'ADV':
                                    eadvmod_pairs.append(j.text+' '+token.text)
                                if j.dep_ is 'neg' and j.pos_ is 'ADV':
                                    eneg_pairs.append(j.text+' '+token.text)
                            for j in token.rights:
                                if j.dep_ is 'advmod'and j.pos_ is 'ADV':
                                    eadvmod_pairs.append(token.text+' '+j.text)
                        if token.pos_ is 'ADJ':
                            for j in token.rights:
                                if j.dep_ is 'xcomp':
                                    for k in j.lefts:
                                        if k.dep_ is 'aux':
                                            excomp_pairs.append(token.text+' '+k.text+' '+j.text)
            pairs = list(set(amod_pairs+advmod_pairs+neg_pairs+xcomp_pairs))
            print(pairs)
            epairs = list(set(eamod_pairs+eadvmod_pairs+eneg_pairs+excomp_pairs))
            for i in range(len(pairs)):
                if len(compound_pairs)!=0:
                    for comp in compound_pairs:
                        mtch = re.search(re.escape(comp[1]),re.escape(pairs[i]))
                        if mtch is not None:
                            pairs[i] = pairs[i].replace(mtch.group(),comp[0])
            for i in range(len(epairs)):
                if len(ecompound_pairs)!=0:
                    for comp in ecompound_pairs:
                        mtch = re.search(re.escape(comp[1]),re.escape(epairs[i]))
                        if mtch is not None:
                            epairs[i] = epairs[i].replace(mtch.group(),comp[0])

        aspect_terms.append(pairs)
        comp_terms.append(compound_pairs)
        easpect_terms.append(epairs)
        ecomp_terms.append(ecompound_pairs)
        enemy.append(enemlist)
    data['compound_nouns'] = comp_terms
    data['aspect_keywords'] = aspect_terms
    data['competition'] = enemy
    data['competition_comp_nouns'] = ecomp_terms
    data['competition_aspects'] = easpect_terms
    return data

In [7]:
data.shape

(242, 2)

In [8]:
data1 = get_aspect_adj(data)

  0%|                                                                                          | 0/242 [00:00<?, ?it/s]

[]
[]
['big bottle', 'Indian brand', 'LONG time', 'gradually improved', 'single user']


  1%|█                                                                                 | 3/242 [00:00<00:14, 16.67it/s]

["n't need", 'oily scalp', 'naturally straight hair', 'straight hair', 'use afterward', 'lathers easily']


  2%|█▎                                                                                | 4/242 [00:00<00:17, 13.23it/s]

...

 98%|██████████████████████████████████████████████████████████████████████████████▎ | 237/242 [00:09<00:00, 25.58it/s]

['last long', 'mild scent', 'Great buy']
[]
['very nice Product', 'nice product', 'nice Product']


 99%|███████████████████████████████████████████████████████████████████████████████▎| 240/242 [00:10<00:00, 24.65it/s]

[]
[]


100%|████████████████████████████████████████████████████████████████████████████████| 242/242 [00:10<00:00, 23.76it/s]


In [9]:
my_df = pd.DataFrame(columns = ['row_index','aspect_adj_string','pos_score','neg_score','compound_score','neutral_score'])
my_df

,row_index,aspect_adj_string,pos_score,neg_score,compound_score,neutral_score


In [10]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()
my_df = pd.DataFrame(columns = ['row_index','aspect_adj_string','pos_score','neg_score','compound_score','neutral_score'])
global_sentiment = {}
for aspect_adj_tuple in data1.aspect_keywords.iteritems():
    row_sentiment = {}
    for text in aspect_adj_tuple[1]:
        sentiment_array = []
        polarity = analyser.polarity_scores(text)
#         print(polarity)
#         sentiment_array.append(polarity['pos'])
#         sentiment_array.append(polarity['neg'])
#         row_sentiment[text] = sentiment_array
        my_df = my_df.append({'row_index':aspect_adj_tuple[0],'aspect_adj_string':text,'pos_score':polarity['pos'],'neg_score':polarity['neg'],'compound_score':polarity['compound'],'neutral_score':polarity['neu']},ignore_index=True)
#     global_sentiment[aspect_adj_tuple[0]] = row_sentiment
    
# print(global_sentiment)

In [11]:
#Filtering the data to remove the non-sentimental/garbage/gray sentiments from entire dataframe
final_df = my_df[((my_df['compound_score']>0.1) | (my_df['compound_score']< -0.1))]

In [12]:
#verificaction on filtered data
final_df[(final_df['compound_score']<0) & (final_df['pos_score']> final_df['neg_score'])]

,row_index,aspect_adj_string,pos_score,neg_score,compound_score,neutral_score


In [13]:
final_df['final_sentiment'] = ['pos' if i>0 else 'neg' for i in final_df['compound_score']]

C:\Users\Dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [14]:
import spacy

nlp = spacy.load("en_core_web_sm")

for row_txt in final_df.aspect_adj_string.iteritems():
    doc = nlp(row_txt[1])
    adj = []
    aspect = []
    for token in doc:

        if (token.tag_ == 'ADV') or (token.tag_ == 'ADJ') or (token.tag_ == 'UH') or (token.tag_ == 'JJS') or (token.tag_ == 'RB') or (token.tag_ == 'JJ') :
            adj.append(token.text)
            
        elif (token.tag_ == 'NN') or (token.tag_ == 'NNP') or (token.tag_ == 'VBG') or (token.tag_ == 'VB')  or (token.tag_ == 'VBN') or (token.tag_ == 'NNS'):
            aspect.append(token.text)

    if (len(adj) == len(aspect)) & (len(adj)>0):
        final_df.at[row_txt[0],'adj']= adj
        final_df.at[row_txt[0],'aspect']= aspect
    elif len(adj) > len(aspect):
        final_df.at[row_txt[0],'adj']= adj[:len(aspect)]
        final_df.at[row_txt[0],'aspect']= aspect
    elif  len(adj) < len(aspect):
        final_df.at[row_txt[0],'adj']= adj
        final_df.at[row_txt[0],'aspect']= aspect[:len(adj)]
    else:
        final_df.at[row_txt[0],'adj']= adj
        final_df.at[row_txt[0],'aspect']= aspect
            

C:\Users\Dell\Anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\Dell\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [15]:
final_df.to_csv("C:/Users/Dell/Pictures/senti/afteraspect.csv")

In [16]:
final_df.head()

,row_index,aspect_adj_string,pos_score,neg_score,compound_score,neutral_score,final_sentiment,adj,aspect
3,2,gradually improved,0.756,0.0,0.4767,0.244,pos,gradually,improved
7,3,naturally straight hair,0.487,0.0,0.2263,0.513,pos,[naturally],[hair]
8,3,straight hair,0.655,0.0,0.2263,0.345,pos,[straight],[hair]
10,3,lathers easily,0.706,0.0,0.3400,0.294,pos,[easily],[lathers]
14,7,Nice product,0.737,0.0,0.4215,0.263,pos,[Nice],[product]


In [17]:
thirdreg=final_df[['row_index','aspect_adj_string','final_sentiment']]

In [18]:
thirdreg.head()

,row_index,aspect_adj_string,final_sentiment
3,2,gradually improved,pos
7,3,naturally straight hair,pos
8,3,straight hair,pos
10,3,lathers easily,pos
14,7,Nice product,pos


In [19]:
geodata_check    = pd.merge(data,thirdreg["row_index"],on = "row_index",how = "left",indicator = True)
geonew_check     = geodata_check[geodata_check["_merge"] != 'both']


In [20]:
neglected=geonew_check [['row_index','Review']]

In [21]:

neglected['Review']= neglected['Review'].str.replace("[^a-zA-Z#]", " ")
neglected['Review']=neglected['Review'].str.lower()


C:\Users\Dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [22]:
#nltk vader lexicon 
final_sentiment=[]
for i in neglected['Review'].values:
    try:
        senti=SentimentIntensityAnalyzer()
        analysis=senti.polarity_scores(i)
        final_sentiment.append(analysis['compound'])
    except:
        final_sentiment.append(0)
neglected['final_sentiment']=final_sentiment

pos=neglected[['row_index','Review','final_sentiment']][neglected.final_sentiment>0]
neg=neglected[['row_index','Review','final_sentiment']][neglected.final_sentiment<0]
hpos=neglected[['row_index','Review','final_sentiment']][neglected.final_sentiment>0.75]
hneg=neglected[['row_index','Review','final_sentiment']][neglected.final_sentiment<-0.25]

#Converting the polarity values from continuous to categorical
neglected['final_sentiment'][neglected.final_sentiment==0]= 0
neglected['final_sentiment'][neglected.final_sentiment > 0]= 1
neglected['final_sentiment'][neglected.final_sentiment< 0]= -1


C:\Users\Dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Dell\Anaconda3\lib\site-packages\pandas\core\generic.py:8682: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
C:\User

In [23]:
neglected.head()

,row_index,Review,final_sentiment
0,0,noting special ok product,1.0
1,1,review good,1.0
6,4,damaged unexpectedlly one bottle is damaged,-1.0
7,5,not entirely ayurvedic though all these ...,-1.0
8,6,good for hair loss it s good for hair loss,1.0


In [24]:

value = {1.0:'pos',-1.0:'neg',0.0:'neu'} 
neglected.final_sentiment = [value[item] for item in neglected.final_sentiment]  

C:\Users\Dell\Anaconda3\lib\site-packages\pandas\core\generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [25]:
neglected.head()

,row_index,Review,final_sentiment
0,0,noting special ok product,pos
1,1,review good,pos
6,4,damaged unexpectedlly one bottle is damaged,neg
7,5,not entirely ayurvedic though all these ...,neg
8,6,good for hair loss it s good for hair loss,pos


In [26]:
#import RAKE
!pip install rake_nltk
from rake_nltk import Rake
import nltk
nltk.download('stopwords')
from rake_nltk import Rake
from nltk.corpus import stopwords 
from rake_nltk import Rake
from nltk.corpus import stopwords 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [27]:

aspect_adj_string =[]
for i in neglected['Review'].values:
    try:
        r= Rake()
        r = Rake(min_length=1, max_length=3)
        a=r.extract_keywords_from_text(i)
        b=r.get_ranked_phrases()[:3]
        aspect_adj_string.append(b)
    except:
        aspect_adj_string.append(0)
neglected['aspect_adj_string']=aspect_adj_string


C:\Users\Dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [43]:
neglected.sample(10)
#neglected=[['row_index','final_sentiment','aspect_adj_string','Review']]
#thirdreg=[['row_index','Review','final_sentiment','aspect_adj_string']]

,row_index,Review,final_sentiment,aspect_adj_string
32,25,nice nice,pos,[nice nice]
230,188,good good,pos,[good good]
104,82,love this product i use this product form fou...,pos,"[use, product, love]"
267,223,too good awesome for dry n frizzy hair,pos,[good awesome]
260,216,worth of money gd product my hairfall almost ...,pos,"[money gd product, hairfall almost gone, worth]"
262,218,shampoo bottle leaked inside the package sh...,neg,[package]
45,36,doesnot suit me my hair fall started even more,neu,[doesnot suit]
56,45,keeps hair soft keeps hair soft but yes hair ...,pos,"[yes hair fall, decreased]"
112,90,good good,pos,[good good]
25,20,NaN,neu,0


In [29]:
thirdreg.sample(10)

,row_index,aspect_adj_string,final_sentiment
228,171,nice shampoo,pos
208,151,best shampoo,pos
243,186,Good product,pos
43,26,Good product,pos
176,130,best products,pos
306,233,stopped completely,neg
199,144,Good product,pos
265,201,Nice product,pos
112,66,Good life,pos
239,177,Good product,pos


In [30]:
#geodata_check    = pd.merge(thirdreg,data["Review"],on = "row_index",how = "left",indicator = True)
#geonew_check     = geodata_check[geodata_check["_merge"] != 'both']
initial.head(5)

,row_index,Review
0,0,Noting Special. Ok Product.
1,1,Review. Good
2,2,"Indian brand keeping to its name, best for me!..."
3,3,Satisfied. Bought it for INR 152 during the Am...
4,4,Damaged. Unexpectedlly... One bottle is damaged


In [31]:
neglected.head()

,row_index,Review,final_sentiment,aspect_adj_string
0,0,noting special ok product,pos,[]
1,1,review good,pos,[review good]
6,4,damaged unexpectedlly one bottle is damaged,neg,[damaged]
7,5,not entirely ayurvedic though all these ...,neg,"[entirely ayurvedic though, called ayurvedic s..."
8,6,good for hair loss it s good for hair loss,pos,"[hair loss, good]"


In [34]:
neglected.to_csv("C:/Users/Dell/Pictures/senti/neglectedfinal.csv")

In [35]:
thirdreg.to_csv("C:/Users/Dell/Pictures/senti/thirdregfinal.csv")

In [36]:
initial.head()

,row_index,Review
0,0,Noting Special. Ok Product.
1,1,Review. Good
2,2,"Indian brand keeping to its name, best for me!..."
3,3,Satisfied. Bought it for INR 152 during the Am...
4,4,Damaged. Unexpectedlly... One bottle is damaged


In [38]:
thirdreg.head()

,row_index,aspect_adj_string,final_sentiment
3,2,gradually improved,pos
7,3,naturally straight hair,pos
8,3,straight hair,pos
10,3,lathers easily,pos
14,7,Nice product,pos


In [41]:
#geodata_check    = pd.merge(thirdreg,data["Review"],on = "row_index",how = "left",indicator = True)
#geonew_check     = geodata_check[geodata_check["_merge"] != 'both']
neglected.columns


Index(['row_index', 'Review', 'final_sentiment', 'aspect_adj_string'], dtype='object')

In [58]:
customized=neglected[['row_index','aspect_adj_string','final_sentiment']]
customized.head()
#customized.to_csv("C:/Users/Dell/Pictures/senti/customizedfinal.csv")

,row_index,aspect_adj_string,final_sentiment
0,0,[],pos
1,1,[review good],pos
6,4,[damaged],neg
7,5,"[entirely ayurvedic though, called ayurvedic s...",neg
8,6,"[hair loss, good]",pos


In [57]:
thirdreg.head()
#thirdreg.to_csv("C:/Users/Dell/Pictures/senti/thirdreg.csv")
thirdreg.head()

,row_index,aspect_adj_string,final_sentiment
3,2,gradually improved,pos
7,3,naturally straight hair,pos
8,3,straight hair,pos
10,3,lathers easily,pos
14,7,Nice product,pos


In [54]:
frames = [customized,thirdreg]
customizedalgorithm = pd.concat(frames)

In [48]:
customized.to_csv("C:/Users/Dell/Pictures/senti/customized rule based algorithm final op.csv")

In [49]:
result = pd.concat([customized,thirdreg], axis=1, sort=False)